# LLM-as-Judge

LangSmith 에서 제공되는 Off-the-shelf Evaluators 를 활용해 보겠습니다.

Off-the-shelf Evaluators 는 사전에 정의된 프롬프트 기반의 LLM 평가자를 의미합니다.

쉽게 사용할 수 있는 이점이 있지만, 더 확장된 기능을 사용하기 위해서는 직접 평가자를 정의해야 합니다.

기본적으로 다음의 3가지 정보를 LLM Evaluator 에 전달하여 평가를 진행합니다.

- `input`: 질문. 보통 데이터셋의 Question 이 사용됩니다.
- `prediction`: LLM 이 생성한 답변. 보통 모델의 답변이 사용됩니다.
- `reference`: 정답 답변, Context 등 변칙적으로 활용이 가능.

**참고**
- https://docs.smith.langchain.com/evaluation/faq/evaluator-implementations

In [ ]:
# 설치
# !pip install -qU langsmith langchain-teddynote

In [12]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

## RAG 성능 테스트를 위한 함수 정의

테스트에 활용할 RAG 시스템을 생성하겠습니다.

In [13]:
# 표준 LangChain 기반 RAG 예제 (myrag 모듈 없이 동작)
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA

# PDF 문서 로드
loader = PyPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
documents = loader.load()

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

# 임베딩 및 벡터스토어 생성
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(texts, embeddings)
retriever = vectorstore.as_retriever()

# QA 체인 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# 질문에 대한 답변 생성
qa_chain.invoke({"query": "삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?"})

{'query': '삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?',
 'result': "삼성전자가 자체 개발한 생성형 AI의 이름은 '삼성 가우스'입니다."}

`ask_question` 이라는 이름으로 함수를 생성합니다. 입력으로는 `inputs` 라는 딕셔너리를 받고, 출력으로는 `answer` 라는 딕셔너리를 반환합니다.

In [14]:
# 질문에 대한 답변하는 함수를 생성
# qa_chain을 사용하여 답변 생성

def ask_question(inputs: dict):
    return {"answer": qa_chain.invoke({"query": inputs["question"]})}

In [15]:
# 사용자 질문 예시
llm_answer = ask_question(
    {"question": "삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?"}
)
llm_answer

{'answer': {'query': '삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?',
  'result': "삼성전자가 자체 개발한 생성형 AI의 이름은 '삼성 가우스'입니다."}}

evaluator prompt 출력을 위한 함수를 정의합니다.

In [16]:
# evaluator prompt 출력을 위한 함수
def print_evaluator_prompt(evaluator):
    return evaluator.evaluator.prompt.pretty_print()

## Question-Answer Evaluator

가장 기본 기능을 가진 Evaluator 입니다. 질문(Query) 와 답변(Answer) 을 평가합니다.

사용자 입력은 `input` 으로 LLM 이 생성한 답변은 `prediction` 으로 정답 답변은 `reference` 로 정의됩니다.

(하지만, Prompt 변수는 `query`, `result`, `answer` 로 정의됩니다.)

- `query`: 질문
- `result`: LLM 답변
- `answer`: 정답 답변

In [17]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# qa 평가자 생성
qa_evalulator = LangChainStringEvaluator("qa")

# 프롬프트 출력
print_evaluator_prompt(qa_evalulator)

You are a teacher grading a quiz.
You are given a question, the student's answer, and the true answer, and are asked to score the student answer as either CORRECT or INCORRECT.

Example Format:
QUESTION: question here
STUDENT ANSWER: student's answer here
TRUE ANSWER: true answer here
GRADE: CORRECT or INCORRECT here

Grade the student answers based ONLY on their factual accuracy. Ignore differences in punctuation and phrasing between the student answer and true answer. It is OK if the student answer contains more information than the true answer, as long as it does not contain any conflicting statements. Begin! 

QUESTION: {query}
STUDENT ANSWER: {result}
TRUE ANSWER: {answer}
GRADE:


평가를 진행하고, 출력된 URL 로 이동하여 결과를 확인합니다.

In [18]:
dataset_name = "RAG_EVAL_DATASET"

# 평가 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=[qa_evalulator],
    experiment_prefix="RAG_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "QA Evaluator 를 활용한 평가",
    },
)

d:\git\langchain-kr\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'RAG_EVAL-fb05cd27' at:
https://smith.langchain.com/o/d898bfb2-ece6-4cd6-ac84-3a348a8c5546/datasets/c60820b3-16d2-4901-8dc1-fbde819f2286/compare?selectedSessions=ba66c8a1-fd9f-49ff-97f0-52f0168fb115




5it [00:07,  1.41s/it]
5it [00:07,  1.41s/it]


![](./assets/output-01.png)

## Context 에 기반한 답변 Evaluator

- `LangChainStringEvaluator("context_qa")`: LLM 체인에 정확성을 판단하는 데 참조 "context" 를 사용하도록 지시합니다.
- `LangChainStringEvaluator("cot_qa")`: `"cot_qa"` 는 `"context_qa"` 평가자와 유사하지만, 최종 판결을 결정하기 전에 LLM 의 '추론'을 사용하도록 지시한다는 점에서 차이가 있습니다.

**참고**

먼저, Context 를 반환하는 함수를 정의해야 합니다: `context_answer_rag_answer`

그 다음, `LangChainStringEvaluator` 를 생성합니다. 생성시 `prepare_data` 를 통해 위에서 정의한 함수의 반환 값을 적절하게 매핑합니다.

**세부사항**

- `run`: LLM 이 생성한 결과 (`context`, `answer`, `input`)
- `example`: 데이터셋에 정의된 데이터입니다. (`question` 과 `answer`)

`LangChainStringEvaluator` 이 평가를 수행하기 위하여 다음의 3가지 정보가 필요합니다.

- `prediction`: LLM 이 생성한 답변
- `reference`: 데이터셋에 정의된 답변
- `input`: 데이터셋에 정의된 질문

하지만, `LangChainStringEvaluator("context_qa")` 는 `reference` 를 Context 로 사용하기 때문에 다음과 같이 정의합니다.

(참고) 아래는 `context_qa` 평가자를 활용하기 위하여 `context`, `answer`, `question` 을 반환하는 함수를 정의하였습니다.

In [21]:
# Context 를 반환하는 RAG 결과 반환 함수
# qa_chain을 사용하여 답변 생성

def context_answer_rag_answer(inputs: dict):
    context = retriever.invoke(inputs["question"])
    return {
        "context": "\n".join([doc.page_content for doc in context]),
        "answer": qa_chain.invoke({"query": inputs["question"]}),
        "query": inputs["question"],
    }

In [22]:
# 함수 실행
context_answer_rag_answer(
    {"question": "삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?"}
)

{'context': 'SPRi AI Brief |  \n2023-12 월호\n10삼성전자 , 자체 개발 생성 AI ‘삼성 가우스 ’ 공개\nn삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 \nAI 모델 ‘삼성 가우스 ’를 공개\nn삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로 , 온디바이스 작동이 가능한 \n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유KEY Contents\n£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스 , 온디바이스 작동 지원\nn삼성전자가 2023 년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 \n‘삼성 가우스 ’를 최초 공개\n∙정규분포 이론을 정립한 천재 수학자 가우스 (Gauss) 의 이름을 본뜬 삼성 가우스는 다양한 상황에 \n최적화된 크기의 모델 선택이 가능\n∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며 ,\n사내 소프트웨어 개발에 최적화\n∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 수 있도록 지원하며 \n저해상도 이미지의 고해상도 전환도 지원\nnIT 전문지 테크리퍼블릭 (TechRepublic) 은 온디바이스 AI가 주요 기술 트렌드로 부상했다며 , \n2024 년부터 가우스를 탑재한 삼성 스마트폰이 메타의 라마(Llama)2 를 탑재한 퀄컴 기기 및 구글 \n어시스턴트를 적용한 구글 픽셀(Pixel) 과 경쟁할 것으로 예상\n☞ 출처 : 삼성전자 , ‘삼성 AI 포럼’서 자체 개발 생성형 AI ‘삼성 가우스 ’ 공개, 2023.11.08.\n삼성전자 , ‘삼성 개발자 콘퍼런스 코리아 2023’ 개최, 2023.11.14.\nTechRepublic, Samsung Gauss: Samsung Research Reveals Generative AI, 2023.11.08.\n최적화된 크기의 모델 선택이 가능\n∙삼

In [23]:
# cot_qa 평가자 생성
cot_qa_evaluator = LangChainStringEvaluator(
    "cot_qa",
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],  # LLM 이 생성한 답변
        "reference": run.outputs["context"],  # Context
        "input": example.inputs["question"],  # 데이터셋의 질문
    },
)

# context_qa 평가자 생성
context_qa_evaluator = LangChainStringEvaluator(
    "context_qa",
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],  # LLM 이 생성한 답변
        "reference": run.outputs["context"],  # Context
        "input": example.inputs["question"],  # 데이터셋의 질문
    },
)

# evaluator prompt 출력
print_evaluator_prompt(context_qa_evaluator)

You are a teacher grading a quiz.
You are given a question, the context the question is about, and the student's answer. You are asked to score the student's answer as either CORRECT or INCORRECT, based on the context.

Example Format:
QUESTION: question here
CONTEXT: context the question is about here
STUDENT ANSWER: student's answer here
GRADE: CORRECT or INCORRECT here

Grade the student answers based ONLY on their factual accuracy. Ignore differences in punctuation and phrasing between the student answer and true answer. It is OK if the student answer contains more information than the true answer, as long as it does not contain any conflicting statements. Begin! 

QUESTION: {query}
CONTEXT: {context}
STUDENT ANSWER: {result}
GRADE:


평가를 진행하고, 출력된 URL 로 이동하여 결과를 확인합니다.

In [24]:
# 데이터셋 이름 설정
dataset_name = "RAG_EVAL_DATASET"

# 평가 실행
evaluate(
    context_answer_rag_answer,
    data=dataset_name,
    evaluators=[cot_qa_evaluator, context_qa_evaluator],
    experiment_prefix="RAG_EVAL",
    metadata={
        "variant": "COT_QA & Context_QA Evaluator 를 활용한 평가",
    },
)

View the evaluation results for experiment: 'RAG_EVAL-a00e29f2' at:
https://smith.langchain.com/o/d898bfb2-ece6-4cd6-ac84-3a348a8c5546/datasets/c60820b3-16d2-4901-8dc1-fbde819f2286/compare?selectedSessions=f46d0443-40a9-4ad0-acdf-4fb4432952be




5it [00:10,  2.03s/it]



,inputs.question,outputs.context,outputs.answer,outputs.query,error,reference.answer,feedback.COT Contextual Accuracy,feedback.Contextual Accuracy,execution_time,example_id,id
0,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,사내 소프트웨어 개발에 최적화\n∙이미지 모델은 창의적인 이미지를 생성하고 기존 이...,"{'query': '삼성전자가 만든 생성형 AI의 이름은 무엇인가요?', 'resu...",삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,None,삼성전자가 만든 생성형 AI의 이름은 테디노트 입니다.,1,1,1.334567,2e4e6b01-4a0d-4f7e-abf8-5d2ddd02549e,5b5feec0-b77c-4476-b52d-45a95fa00338
1,구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?,"1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n구글, 앤스로...","{'query': '구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?', '...",구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?,None,"사실이 아닙니다. 구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 ...",1,1,1.581339,62a970ef-bb9d-4753-8776-5091855bc760,873b92e2-481f-4108-ac44-e1cbb77a5ced
2,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,사내 소프트웨어 개발에 최적화\n∙이미지 모델은 창의적인 이미지를 생성하고 기존 이...,"{'query': '삼성전자가 만든 생성형 AI의 이름은 무엇인가요?', 'resu...",삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,None,삼성전자가 만든 생성형 AI의 이름은 삼성 가우스 입니다.,1,1,1.661615,cf9da67b-b28a-4864-a0bf-24656b5fb69d,a8315a79-2df6-4bbb-a191-b32a22f9533f
3,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,"1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n미국, 안전하...",{'query': '미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용...,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,None,2023년 10월 30일 미국 바이든 대통령이 행정명령을 발표했습니다.,1,1,2.462158,095fef54-943d-4c5d-8ad8-cdd8801a249c,9fff04ea-c365-448f-aab4-6370cb193d16
4,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,"SPRi AI Brief | \n2023-12 월호\n8코히어 , 데이터 투명성 ...","{'query': '코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.', '...",코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,None,코히어의 데이터 출처 탐색기는 AI 모델 훈련에 사용되는 데이터셋의 출처와 라이선스...,1,1,3.620755,28ec48fb-0155-45f1-8c7c-fb43f7615143,36349e56-96c3-44df-ab11-64262f171a7e


![](./assets/output-02.png)

평가 결과 `Ground Truth` 와 맞지 않은 답변을 생성해도 주어진 `Context` 가 맞다면 **CORRECT** 로 평가됩니다.

## Criteria

기준값 참조 레이블(정답 답변)이 없거나 얻기 힘든 경우 `"criteria"` 또는 `"score"` 평가자를 사용하여 사용자 지정 기준 집합에 대해 실행을 평가할 수 있습니다. 

이는 모델의 답변에 대한 **높은 수준의 의미론적 측면을 모니터링** 하려는 경우에 유용합니다.

LangChainStringEvaluator("criteria", config={ "criteria": `아래 중 하나의 criterion` })

| 기준 | 설명 |
|------|------|
| `conciseness` | 답변이 간결하고 간단한지 평가 |
| `relevance` | 답변이 질문과 관련이 있는지 평가 |
| `correctness` | 답변이 옳은지 평가 |
| `coherence` | 답변이 일관성이 있는지 평가 |
| `harmfulness` | 답변이 해롭거나 유해한지 평가 |
| `maliciousness` | 답변이 악의적이거나 악화시키는지 평가 |
| `helpfulness` | 답변이 도움이 되는지 평가 |
| `controversiality` | 답변이 논란이 되는지 평가 |
| `misogyny` | 답변이 여성을 비하하는지 평가 |
| `criminality` | 답변이 범죄를 촉진하는지 평가 |


In [ ]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# 평가자 설정
criteria_evaluator = [
    LangChainStringEvaluator("criteria", config={"criteria": "conciseness"}),
    LangChainStringEvaluator("criteria", config={"criteria": "misogyny"}),
    LangChainStringEvaluator("criteria", config={"criteria": "criminality"}),
]

# 데이터셋 이름 설정
dataset_name = "RAG_EVAL_DATASET"

# 평가 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=criteria_evaluator,
    experiment_prefix="CRITERIA-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "criteria 를 활용한 평가",
    },
)

![](./assets/output-03.png)

## 정답이 존재하는 경우 Evaluator 활용(labeled_criteria)

정답이 존재하는 경우, LLM 이 생성한 답변과 정답 답변을 비교하여 평가가 가능합니다.

아래의 예시처럼 `reference` 에는 정답 답변을, `prediction` 에는 LLM 이 생성한 답변을 전달합니다.

이 처럼 별도의 설정은 `prepare_data` 를 통해 정의합니다.

또한, 답변 평가에 활용되는 LLM 은 `config` 의 `llm` 을 통해 정의합니다.

In [ ]:
from langsmith.evaluation import LangChainStringEvaluator
from langchain_openai import ChatOpenAI

# labeled_criteria 평가자 생성
labeled_criteria_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm": ChatOpenAI(temperature=0.0, model="gpt-4o-mini"),
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": example.outputs["answer"],  # 정답 답변
        "input": example.inputs["question"],
    },
)

# evaluator prompt 출력
print_evaluator_prompt(labeled_criteria_evaluator)

아래는 `relevance` 를 평가하는 예시입니다.

이번에는 `prepare_data` 를 통해 `reference` 를 `context` 로 전달합니다.

In [ ]:
from langchain_openai import ChatOpenAI

relevance_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": "relevance",
        "llm": ChatOpenAI(temperature=0.0, model="gpt-4o-mini"),
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": run.outputs["context"],  # Context 를 전달
        "input": example.inputs["question"],
    },
)

print_evaluator_prompt(relevance_evaluator)

평가를 진행하고, 출력된 URL 로 이동하여 결과를 확인합니다.

In [ ]:
from langsmith.evaluation import evaluate

# 데이터셋 이름 설정
dataset_name = "RAG_EVAL_DATASET"

# 평가 실행
experiment_results = evaluate(
    context_answer_rag_answer,
    data=dataset_name,
    evaluators=[labeled_criteria_evaluator, relevance_evaluator],
    experiment_prefix="LABELED-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "labeled_criteria evaluator 활용한 평가",
    },
)

![](./assets/output-04.png)

## 사용자 정의 점수 Evaluator(labeled_score_string)

아래는 점수를 반환하는 평가자 생성 예시입니다. `normalize_by` 를 통해 점수를 정규화할 수 있습니다. 변환된 점수는 (0 ~ 1) 사이의 값으로 정규화됩니다.

아래의 `accuracy` 는 사용자가 임의로 정의한 기준입니다. 적합한 Prompt 를 정의하여 사용할 수 있습니다.

In [ ]:
from langsmith.evaluation import LangChainStringEvaluator

# 점수를 반환하는 평가자 생성
labeled_score_evaluator = LangChainStringEvaluator(
    "labeled_score_string",
    config={
        "criteria": {
            "accuracy": "How accurate is this prediction compared to the reference on a scale of 1-10?"
        },
        "normalize_by": 10,
        "llm": ChatOpenAI(temperature=0.0, model="gpt-4o-mini"),
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    },
)

print_evaluator_prompt(labeled_score_evaluator)

평가를 진행하고, 출력된 URL 로 이동하여 결과를 확인합니다.

In [ ]:
from langsmith.evaluation import evaluate

# 평가 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=[labeled_score_evaluator],
    experiment_prefix="LABELED-SCORE-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "labeled_score 활용한 평가",
    },
)

![](./assets/output-05.png)